In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [23]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shantanudhakadd/bank-customer-churn-prediction")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/shantanudhakadd/bank-customer-churn-prediction/versions/1


In [24]:
df=pd.read_csv(path + "/Churn_Modelling.csv" , encoding='ISO-8859-1')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [25]:
df.isna().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [26]:
df=df.drop('RowNumber',axis=1)
df=df.drop('CustomerId',axis=1)
df=df.drop('Surname',axis=1)

In [27]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [28]:
X=df.drop('Exited',axis=1)
y=df['Exited'].copy()

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
cat_cols=['Geography','Gender','NumOfProducts','HasCrCard','IsActiveMember']
num_cols=['CreditScore','Age','Tenure','Balance','EstimatedSalary']

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

cat_pipeline=make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

num_pipeline=make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

full_pipeline=ColumnTransformer([
        ('cat',cat_pipeline,cat_cols),
        ('num',num_pipeline,num_cols)]
)
X_transformed = pd.DataFrame(full_pipeline.fit_transform(X), columns = full_pipeline.get_feature_names_out())
X_train_transformed = pd.DataFrame(full_pipeline.fit_transform(X_train), columns = full_pipeline.get_feature_names_out())
X_test_transformed = pd.DataFrame(full_pipeline.fit_transform(X_test), columns = full_pipeline.get_feature_names_out())


In [40]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier()
rnd_clf.fit(X_train_transformed,y_train)
print("RandomForestClassifier_score:", rnd_clf.score(X_test_transformed,y_test))

from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(X_train_transformed,y_train)
log_reg.score(X_test_transformed,y_test)
print("LogisticRegression_score:" ,log_reg.score(X_test_transformed,y_test))

from sklearn.tree import DecisionTreeClassifier
tree_clf=DecisionTreeClassifier()
tree_clf.fit(X_train_transformed,y_train)
print("DecisionTreeClassifier_score:" ,tree_clf.score(X_test_transformed,y_test))


RandomForestClassifier_score: 0.866
LogisticRegression_score: 0.8425
DecisionTreeClassifier_score: 0.7855


#Best Params

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from scipy.stats import randint

poly_scaler_ridge_pipeline = Pipeline([
    ("model", RandomForestClassifier(random_state = 42)),
])

param_distribs = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [10, 30, 50],
}
grid_search = GridSearchCV(poly_scaler_ridge_pipeline, param_grid=param_distribs, cv=3,scoring='f1_macro')

grid_search.fit(X_transformed, y)

grid_search.best_params_

{'model__max_depth': 30, 'model__n_estimators': 200}

#Bagging and Pasting

In [59]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_depth = 30),n_estimators = 200,
    max_samples = 100,bootstrap = True,n_jobs = -1
)
bag_clf.fit(X_train_transformed,y_train)
print("Bagging_score:",bag_clf.score(X_test_transformed,y_test))


Bagging_score: 0.854


In [58]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_depth = 30),n_estimators = 200,
    max_samples = 100,bootstrap = False,n_jobs = -1
)
bag_clf.fit(X_train_transformed,y_train)
print("Pasting_score:",bag_clf.score(X_test_transformed,y_test))

Pasting_score: 0.8575


#AdaBoosting

In [65]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 30),n_estimators = 200,learning_rate = 0.5)
ada_clf.fit(X_train_transformed,y_train)
print("AdaBoosting_score:",ada_clf.score(X_test_transformed,y_test))

AdaBoosting_score: 0.7875


#GradientBoosting

In [66]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(max_depth=30,
                                 n_estimators=200,
                                 learning_rate=.5,
                                 random_state=42)
gbrt.fit(X_train_transformed, y_train)
print("GradientBoosting_score:",gbrt.score(X_test_transformed,y_test))

GradientBoosting_score: 0.7905


#VotingClassifier

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

voting_clf = VotingClassifier(
    estimators = [('lr',LogisticRegression()),
                  ('rf',RandomForestClassifier()),
                  ('dt',DecisionTreeClassifier())]
)
voting_clf.fit(X_train_transformed,y_train)
print("VotingClassifier_score:",voting_clf.score(X_test_transformed,y_test))


VotingClassifier_score: 0.855
